In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from util_func import *
from test_on_udacity_data import *
from convert_to_full_view_panorama import cluster
from sklearn.cluster import DBSCAN
import os
import time

In [2]:
def nearby_car(cluster, gt_box, k = 1):
    if len(gt_box) == 0:
        return False
    for i in range(len(gt_box)):
        center = np.mean(cluster[:,:2], axis = 0)
        box_center = np.mean(gt_box[i][[0,2],:2], axis = 0 )
        distance =  np.sqrt(np.sum(np.square(center - box_center)))    
        
        box_diagonal = np.sqrt(np.sum(np.square(gt_box[i][0,:2] - gt_box[i][2,:2])))
        # The k parameter is used to control the distance from cluster to box_center
        if distance <= k*box_diagonal:
            return True
        else:
            return False

In [22]:
def is_good_label(lidar, gt_box, k=1):
    '''
    return:
        -1 : if no cluster is found. We better to check the frame by own eyes. Remove this frame from training set
                because there is a gt_box but the training panoramic view is full of zeros.
        0 : if no cluster is near to the gt_box
        if at least one cluster is near to gt_box, the function return the number of near cluster  
    '''
    
    assert len(gt_box) > 0, "Empty groundtruth box"
    
    lidar, labels = cluster(lidar, min_d = 2, min_z = -1.35, max_z = 0.5, max_xrange = 6,
                             max_yrange = 6, min_xrange = 0.5, min_yrange = 0.5,  
                             min_zrange = 0.2, min_points = 15, z_scale = 1,eps = 0.8, min_samples = 3)
    # list of clusters
    clusters = list(set(labels))
    n_clusters = len(clusters)
    
    if n_clusters == 0:
        return -1
    
    n_near_clusters = 0
    
    for i in range(n_clusters):
        cluster_lidar = lidar[labels == clusters[i]]
        if nearby_car(cluster_lidar, gt_box):
            n_near_clusters += 1
            # we can save the car cluster separately in this step
    return n_near_clusters

### Visualize the code

In [26]:
def visualize_each_cluster(lidar, labels, gtbox=None): 
    clusters = list(set(labels))
    n_clusters = len(clusters)
    print('Number of clustering: ', n_clusters)
    for i in range(n_clusters):
        print(i+1,' - cluster: ',clusters[i])
        #viz_mayavi_clustering_lidar(lidar2[labels2 == clusters[i]], labels2[labels2 == clusters[i]])
        print(lidar[labels == clusters[i]].shape)
        if gtbox != None:
            viz_mayavi_with_labels(lidar[labels == clusters[i]], gtbox)
        else:
            viz_mayavi_with_labels(lidar[labels == clusters[i]], [], False)

In [20]:
## We cant dectect any cluster in this frame.  

lidar = np.load('./data/training_didi_data/car_train_edited/suburu_driving_towards_it/lidar/lidar_108.npy')
gtbox = np.load('./data/training_didi_data/car_train_gt_box_edited/suburu_driving_towards_it/gt_boxes3d/gt_boxes3d_108.npy')

viz_mayavi_with_labels(lidar, gtbox) 

In [23]:
print(is_good_label(lidar,gtbox))

-1


In [24]:
## Detect 2 good clusters this frame. Both of them is belong to one car. Increase eps can make them to one cluster   

lidar = np.load('./data/training_didi_data/car_train_edited/suburu_follows_capture/lidar/lidar_228.npy')
gtbox = np.load('./data/training_didi_data/car_train_gt_box_edited/suburu_follows_capture/gt_boxes3d/gt_boxes3d_228.npy')

viz_mayavi_with_labels(lidar, gtbox) 

In [17]:
print(is_good_label(lidar,gtbox))

2


In [27]:
lidar, labels = cluster(lidar, min_d = 2, min_z = -1.4, max_z = 0.5, max_xrange = 6,
                             max_yrange = 6, min_xrange = 0.5, min_yrange = 0.5,  
                             min_zrange = 0.2, min_points = 15, z_scale = 1,eps = 0.8, min_samples = 1)
visualize_each_cluster(lidar, labels, gtbox)

Number of clustering:  2
1  - cluster:  0
(33, 3)


/home/minh/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:9: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


2  - cluster:  1
(36, 3)


In [43]:
## Detect one cluster but the gt_box is bad

lidar = np.load('./data/training_didi_data/car_train_edited/suburu_pulling_to_right/lidar/lidar_181.npy')
gtbox = np.load('./data/training_didi_data/car_train_gt_box_edited/suburu_pulling_to_right/gt_boxes3d/gt_boxes3d_181.npy')

viz_mayavi_with_labels(lidar, gtbox) 

In [44]:
print(is_good_label(lidar,gtbox))
cluster_lidar, labels = cluster(lidar, min_d = 2, min_z = -1.35, max_z = 0.5, max_xrange = 6,
                             max_yrange = 6, min_xrange = 0.5, min_yrange = 0.5,  
                             min_zrange = 0.2, min_points = 15, z_scale = 1,eps = 0.8, min_samples = 1)
visualize_each_cluster(cluster_lidar, labels, gtbox)

0
Number of clustering:  1
1  - cluster:  0
(4829, 3)


/home/minh/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:9: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


#### Detect all bad label and save car_points, not_car_points

In [ ]:
def create_car_training(lidar_dir, gtbox_dir, car_dir, not_car_dir):
    # create folders contains car and not_car lidar points if not exist
    if not os.path.exists(car_dir):
        os.mkdir(car_dir)
    if not os.path.exists(not_car_dir):
        os.mkdir(not_car_dir)
    # list of subfolders containing car lidar
    list_folders = os.listdir(lidar_dir)
    print('Begin converting {} lidar folders.'.format(len(list_folders)))
    start = time.time()
    log = open('log_cars.txt', 'a')
    bad_car_labels = open('bad_car_labels.txt', 'a')
    for folder in list_folders:
        lidar_folder = os.path.join(lidar_dir, folder, 'lidar')
        gtbox_folder = os.path.join(gtbox_dir, folder, 'gt_boxes3d')
        
        car_folder = os.path.join(car_dir, folder)
        not_car_folder = os.path.join(not_car_dir, folder)
        
        if not os.path.exists(car_folder):
            os.mkdir(car_folder)
        if not os.path.exists(not_car_folder):
            os.mkdir(not_car_folder)
        # list of all lidar frame
        list_lidar_files = os.listdir(lidar_folder)
        #list_lidar_files = list_lidar_files[:5]
        #print(list_lidar_files)
        
        print('Begin converting {0} lidar files in folder {1}'.format(len(list_lidar_files), folder))
        
        for file in list_lidar_files:
            # load lidar frame and gt_box
            #print(file)
            lidar = np.load(os.path.join(lidar_folder,file))
            gt_box = np.load(os.path.join(gtbox_folder,'gt_boxes3d'+file[5:]))
            # clustering lidar frame
            #lidar, labels = clustering(lidar, min_d = 2, min_z = -1.4, max_z = 0.5, max_xrange = 1, 
            #       max_yrange = 1, min_zrange = 0.5, min_points = 20, z_scale = 10.,eps = 0.2, min_samples = 3)
            lidar, labels = clustering(lidar, min_d = 2, min_z = -1.4, max_z = 2, max_xrange = 5,
                             max_yrange = 5, min_xrange = 0.5, min_yrange = 0.5,  
                             min_zrange = 0.2, min_points = 50, z_scale = 2,eps = 0.7, min_samples = 3)
            # list of clusters
            clusters = list(set(labels))
            n_clusters = len(clusters)
            if n_clusters == 0:
                log.write(folder+'-'+file + ' num_gtbox:' + str(len(gt_box)) + ' clusters:0\n')
                bad_car_labels.write(folder+'-'+file + ' num_gtbox:' + str(len(gt_box)) + ' clusters:0\n')
            else:
                n_cars = 0
                n_not_cars = 0
                for i in range(n_clusters):
                    cluster = lidar[labels == clusters[i]]
                    if nearby_car(cluster, gt_box):
                        np.save(os.path.join(car_folder,'car_' + file[5:-4] + '_' + str(n_cars) + '.npy'), cluster)
                        n_cars += 1
                    else:
                        np.save(os.path.join(not_car_folder,'not_car_' + file[5:-4]) +'_'+ str(n_not_cars) + '.npy', cluster)
                        n_not_cars += 1
                log.write(folder+'-'+file + ' num_gtbox:' + str(len(gt_box)) +  ' cars:' + str(n_cars) + ' not_cars:' + str(n_not_cars) + '\n')
                if n_cars != len(gt_box):
                    bad_car_labels.write(folder+'-'+file + ' num_gtbox:' + str(len(gt_box)) +  ' cars:' + str(n_cars) + ' not_cars:' + str(n_not_cars) + '\n')
    log.close()
    bad_car_labels.close()
    print('Done creating training data. Total time: ', time.time() - start)